In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import pearsonr
from gershman_sim import *
from gershman_fit import *

import warnings
import multiprocessing as mp
import time
warnings.filterwarnings('ignore')

In [ ]:
# simulate agents
n_trials = 10 
n_blocks = 20
n_agent = 10_000 

t_parameters = np.zeros(shape=(n_agent,2))
all_data = [] 

for agent in range(n_agent):
    beta = np.random.uniform(0,4) 
    gamma = np.random.uniform(0,1)
    t_parameters[agent] = beta,gamma
    sim = gershman_sim(agent,
                       t_parameters[agent],
                       n_blocks,
                       n_trials,
                       probability_to_switch_parameters=0.02,
                       max_change=np.random.choice([2,3,4])
                      )
    df = pd.DataFrame(sim)
    all_data.append(df)
    
df_all_agent = pd.concat(all_data)    
df_all_agent = df_all_agent.reset_index()
df_all_agent = df_all_agent.drop(columns=['index'])  

In [ ]:
x = 4/5

# create a list of our conditions
conditions = [
    (df_all_agent['beta'] <=  x ) ,
    (df_all_agent['beta'] > 1*x ) & (df_all_agent['beta'] <= 2*x ),
    (df_all_agent['beta'] > 2*x ) & (df_all_agent['beta'] <= 3*x ),
    (df_all_agent['beta'] > 3*x ) & (df_all_agent['beta'] <= 4*x ),
    (df_all_agent['beta'] > 4*x ) 
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4] 

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['beta_categorical'] = np.select(conditions, values)

x = 1/5

# create a list of our conditions
conditions = [
    (df_all_agent['gamma'] <=  x ) ,
    (df_all_agent['gamma'] > 1*x ) & (df_all_agent['gamma'] <= 2*x ),
    (df_all_agent['gamma'] > 2*x ) & (df_all_agent['gamma'] <= 3*x ),
    (df_all_agent['gamma'] > 3*x ) & (df_all_agent['gamma'] <= 4*x ),
    (df_all_agent['gamma'] > 4*x ) 
    
    ]

# create a list of the values we want to assign for each condition
values = [0,1,2,3,4] 

# create a new column and use np.select to assign values to it using our lists as arguments
df_all_agent['gamma_categorical'] = np.select(conditions, values)

In [ ]:
df_all_agent[['agent','block','trial',
              'action','reward','beta',
              'gamma','beta_categorical',
              'gamma_categorical']].to_csv('../data/artificial_trainset_10_000.csv',index=False)